In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### load X_train

In [ ]:
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)
coords_fname  = './data/Other/Other/stations_coordinates.csv'
fname = './data/Train/Train/X_station_train.csv'

In [ ]:
def load_X_train(fname,coords_fname) :
    coords = pd.read_csv(coords_fname)
    train = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    # reorganisation des données.
    train['number_sta']=train['number_sta'].astype('category')
    #sort by station, then by date 
    train = train.sort_values(['number_sta','date'])
    method_traitements = ["mean","min","max","std"]
    dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                     ,"hu":method_traitements,"dd":method_traitements,"precip":"sum"}
    Base_obs = train[{"number_sta","date","ff","t","td","hu","dd","precip"}].copy()
    Base_obs.set_index('date',inplace = True)  

    # type de traitement.
    Base_obs_type = Base_obs.groupby('number_sta').resample('D').agg(dict_to_split)
    Base_obs_type.columns = ['_'.join(col).strip() for col in Base_obs_type.columns.values]
    Base_obs_type = Base_obs_type.reset_index(['date','number_sta'])
    Base_obs_type['number_sta'] = Base_obs_type['number_sta'].astype('category')   
    #Select the observations the day before

    Base_obs_type['Prediction'] = Base_obs_type['precip_sum'].shift(1)
    Base_obs_type = Base_obs_type.sort_values(by=["number_sta","date"])
    #del Base_obs['precip']
    #Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})
    del Base_obs_type["precip_sum"]
    date = first_date
    dates = []
    while date <= (last_date - datetime.timedelta(days=1)):
        date += datetime.timedelta(days=1)
        dates.append(date)

    d_dates =  pd.DataFrame(dates, columns = ['date'])
    d_dates['day_index'] = d_dates.index
    y_f = Base_obs_type.merge(d_dates,how="right",on = ["date"])
    y_f = y_f[y_f['date']!=last_date]
    y_f['Id'] = y_f[list(y_f.columns)[0]].astype(str) + '_' + \
                      y_f['day_index'].astype(str) 
    #y_f["number_sta"] = y_f[("number_sta","")]
    y_f = y_f.merge(coords,on=["number_sta"],how="right")
    X_train = y_f.copy()#drop(["day_index"],axis=1)
    X_train["month"] = X_train["date"].dt.month
    X_train["day"] = X_train["date"].dt.day
    X_train.drop(["date"],axis=1,inplace=True)
    return X_train

In [ ]:
train = load_X_train(fname,coords_fname)

### Load Y_test

In [ ]:
fname_test = "./data/Test/Test/X_station_test.csv"
coords_fname_test = './data/Other/Other/stations_coordinates.csv'
coords = pd.read_csv(coords_fname_test)
test = pd.read_csv(fname_test)
test["Id"] =test["Id"].str.rsplit("_",n=1,expand=True)[0]
test.drop(["precip","month"],axis = 1,inplace = True)
method_traitements = ["mean","min","max","std"]
dict_to_split = {"ff":method_traitements,"t":method_traitements,"td":method_traitements
                 ,"hu":method_traitements,"dd":method_traitements}
test["Id"] = test["Id"].astype("category")
Base_obs = test[{"Id","ff","t","td","hu","dd"}].copy()
# type de traitement.
Base_obs_type = Base_obs.groupby('Id').agg(dict_to_split)
Base_obs_type = Base_obs_type.sort_values(by=["Id"])
test = Base_obs_type.copy()
test.columns = ['_'.join(col).strip() for col in test.columns.values]
test = test.reset_index(['Id'])
base_test = pd.read_csv("./data/Test/Test/Baselines/Baseline_observation_test.csv")
test = test.merge(base_test,on=["Id"],how="right")
test[["number_sta","day_index"]] = test["Id"].str.split("_",expand=True)
test["number_sta"] = test["number_sta"].astype(int)
test = test.merge(coords,on="number_sta",how='right')
date = first_date
dates = []
while date <= (last_date - datetime.timedelta(days=1)):
    date += datetime.timedelta(days=1)
    dates.append(date)

d_dates =  pd.DataFrame(dates, columns = ['date'])
d_dates['day_index'] = d_dates.index
d_dates["month"]=d_dates["date"].dt.month
d_dates["day"] = d_dates["date"].dt.day
del d_dates["date"]
test.dropna(subset=["day_index"],inplace=True)
test["day_index"] = test["day_index"].astype(int)
test = test.merge(d_dates,on=["day_index"],how="right")
test = test.dropna(subset=["lat"])

### Load Y

In [ ]:
fname = './data/Train/Train/Y_train.csv'
param = 'Ground_truth'  #weather parameter name in the file ('Ground_truth' about Y and 'Prediction' about baseline)
ytrain= pd.read_csv(fname, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')

In [ ]:
ytrain = ytrain[ytrain["date"]!=last_date].copy()

In [ ]:
### Methodologie drop_all_na
X_full = train.dropna()
Y_full = ytrain.dropna(subset=["Ground_truth"])
#X_full =X_full[X_full.index.isin(Y_full.index)].dropna()
X_full = X_full.sort_values("Id")
Y_full = Y_full.sort_values("Id")
value_list = list(X_full["Id"])
boolean_series = Y_full.Id.isin(value_list)
Y_full = Y_full[boolean_series]
value_list = list(Y_full["Id"])
boolean_series = X_full.Id.isin(value_list)
X_full = X_full[boolean_series]

In [ ]:
X_full = train.copy()

In [ ]:
### Methodologie drop_na from Ytrain
Y_full = ytrain.dropna(subset=["Ground_truth"])

In [ ]:
value_list = list(Y_full["Id"])
boolean_series = X_full.Id.isin(value_list)
X_full = X_full[boolean_series]

In [ ]:
Ximp = X_full.drop(["Id","number_sta","day_index"],axis=1).values#],"lon","lat","height_sta"
Yimp = np.array(Y_full["Ground_truth"])

In [ ]:
### pour l'ensemble test
x_full = test.copy()

In [ ]:
ximp = x_full.drop(["Id","number_sta","day_index"],axis=1).values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xfull = Ximp.copy()

### Application du modèle.

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
### réaliser une imputation des données manquantes.
imputer = KNNImputer(n_neighbors=15)
imputer.fit(ximp)

In [ ]:
ximp = imputer.transform(ximp)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
imp.fit(Ximp)

In [ ]:
ximp=imp.transform(ximp)

### Fonction MAPE

In [ ]:
def MAPE(yp,yt) :
    taille = len(yp)
    mape = 0
    for i in range(taille) :
        mape += abs((yt[i]-yp[i])/(yt[i]+1))
    mape *= 100/taille
    return mape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     Ximp, Yimp, test_size=0.33, random_state=42)

### Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
reg.score(X_train, y_train)

In [ ]:
MAPE(reg.predict(X_train),y_train)

In [ ]:
MAPE(reg.predict(X_test),y_test)

### RandomForestRegression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

In [ ]:
regr.score(X_train,y_train)

In [ ]:
MAPE(regr.predict(X_train),y_train)

In [ ]:
MAPE(regr.predict(Xfull),Yimp)

### Partie submission

In [ ]:
train[train.index.isin(X_full.index)]["Id"]

In [ ]:
y_predicted = list(regr.predict(ximp))
id_predicted = list(test["Id"])

In [ ]:
df_submit = pd.DataFrame()
df_submit["Id"] = id_predicted
df_submit["Prediction"] = y_predicted

In [ ]:
df_submit.to_csv('./working/' + "submission.csv",index=False)

In [ ]:
1